In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import bt2,sys,os,re
from pathlib import Path
sys.path.append('/g/data/tm70/ml0072/COMMON/om3-utils_fork')
from om3utils.esmf_profiling import ESMFProfilingParser
from om3utils.profiling import ProfilingParser,parse_profiling_data
from om3utils.esmf_trace import ESMFTrace, SinglePETTimingNode,MultiPETTimingNode

In [3]:
tmp_parse = ESMFProfilingParser(dirname='/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_new_topo_modify/epbl_1_om5_IO_off/archive/output000')

In [6]:
run_dirs = ['/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_new_topo_modify/epbl_1_om5_IO_off']
varname = 'test'
def getvar(run_dir):
    return os.path.basename(run_dir)
esmf_profile_test = parse_profiling_data(run_dirs,tmp_parse,varname, getvar)

In [7]:
esmf_profile_test

<xarray.Dataset> Size: 58kB
Dimensions:  (region: 230, test: 1)
Coordinates:
  * region   (region) <U53 49kB '[ROF-TO-MED] IPDv05p2b' ... '[ESM0001] IPDv0...
  * test     (test) <U17 68B 'epbl_1_om5_IO_off'
Data variables:
    hits     (test, region) int64 2kB 1 175206 29201 116800 ... 29201 58402 1
    tmin     (test, region) float64 2kB 0.09943 0.02743 ... 1.905e-06 12.68
    tmax     (test, region) float64 2kB 0.3448 0.9077 ... 0.0006489 12.78
    tavg     (test, region) float64 2kB 0.2225 641.8 0.4239 ... 0.07015 12.71
    ttot     (test, region) float64 2kB 106.8 1.54e+05 101.7 ... 16.83 2.013e+04

In [ ]:
selected_regions = select_test_regions(['[ESMF]','[OCN] RunPhase1', 
                                        '[ICE] RunPhase1', '[MED-TO-OCN] RunPhase1', 
                                        '[OCN-TO-MED] RunPhase1','[ICE-TO-MED] RunPhase1',
                                        '[MED-TO-ICE] RunPhase1','[ensemble] Init 1'])
tmplt = 
filtered_dataset = esmf_profile_test.sel(**{varname:tmplt, 'region':regions})

In [11]:
esmf_trace = ESMFTrace(Path('/g/data/tm70/ml0072/COMMON/git_repos/COSIMA_om3-scripts/expts_manager/product1_0.25deg_new_topo_modify/epbl_1_om5_IO_off/archive/output000/traceout'))

In [12]:
esmf_trace

In [13]:
regions = esmf_trace.regions
print("Regions:", regions)

Regions: ['[ROF-TO-MED] IPDv05p2b', 'MED: (med_map_mod:med_map_field_packed)', 'MED:(med_phases_profile)', 'MED:(med_phases_history_write_comp_aux)', 'MED:(med_phases_diag_atm)', '[ATM-TO-MED] IPDv05p1', '[ATM-TO-MED] IPDv05p3', '[ensemble] IPDv05p6', 'MED: (med_map_mod: RouteHandles_init)', '[MED-TO-ICE] IPDv05p2a', '[MED-TO-OCN] IPDv05p4', 'cice_imp_t2u', '[MED] Init 0', '[ATM-TO-MED] Init 0', '[ROF-TO-MED] IPDv05p1', '[MED-TO-ICE] IPDv05p2b', '[ATM-TO-MED] IPDv05p6a', '[MED-TO-ICE] Init 0', '[ESMF]', 'MED: (med_phases_aofluxes_run)', 'cice_init1', 'datm_readLBUB_UB_readpio', '[ATM-TO-MED] IPDv05p4', '[OCN-TO-MED] RunPhase1', '[MED] med_phases_profile', '[MED] med_phases_history_write', '[ROF-TO-MED] IPDv05p5', '[ICE] FinalizePhase1', 'datm_datamode', '[ATM-TO-MED] IPDv05p5', '[MED] med_phases_ocnalb_run', '[ROF] IPDv01p4', 'MED:(med_phases_post_atm) map_atm2ocn', '[MED] med_phases_prep_ice', '[ATM-TO-MED] IPDv05p2a', '[ICE-TO-MED] IPDv05p6a', 'MED: (med_map_mod:med_map_field_packed)

In [14]:
multi_pet_tree = esmf_trace.multiPETTree

# (Assuming MultiPETTimingNode has methods to explore or visualize the tree)
print(multi_pet_tree)

In [16]:
pet0_root = SinglePETTimingNode(_id=0, pet=0, name="root")
pet0_child1 = SinglePETTimingNode(_id=1, pet=0, name="child1")
pet0_child2 = SinglePETTimingNode(_id=2, pet=0, name="child2")

In [17]:
pet0_root.add_child(parentid=0, child=pet0_child1)
pet0_root.add_child(parentid=0, child=pet0_child2)

True

In [18]:
pet0_root.total = 100000  # nanoseconds
pet0_child1.total = 40000
pet0_child2.total = 60000
pet0_root.count = 10
pet0_child1.count = 6
pet0_child2.count = 4

# PET 1 Timing Tree
pet1_root = SinglePETTimingNode(_id=0, pet=1, name="root")
pet1_child1 = SinglePETTimingNode(_id=1, pet=1, name="child1")
pet1_child2 = SinglePETTimingNode(_id=2, pet=1, name="child2")

# Add children to PET 1 tree
pet1_root.add_child(parentid=0, child=pet1_child1)
pet1_root.add_child(parentid=0, child=pet1_child2)

# Set timing statistics for PET 1
pet1_root.total = 120000  # nanoseconds
pet1_child1.total = 50000
pet1_child2.total = 70000
pet1_root.count = 12
pet1_child1.count = 7
pet1_child2.count = 5

In [19]:
multi_pet_root = MultiPETTimingNode()

# Merge PET 0 and PET 1 timing trees into the MultiPETTimingNode
multi_pet_root.merge(pet0_root)
multi_pet_root.merge(pet1_root)

In [20]:
# Access overall timing statistics
print("Total Sum (ns):", multi_pet_root.total_sum)
print("Total Sum (s):", multi_pet_root.total_sum_s)
print("Total Mean (ns):", multi_pet_root.total_mean)
print("Total Mean (s):", multi_pet_root.total_mean_s)
print("Minimum Total (ns):", multi_pet_root.total_min)
print("Minimum Total PET:", multi_pet_root.total_min_pet)
print("Maximum Total (ns):", multi_pet_root.total_max)
print("Maximum Total PET:", multi_pet_root.total_max_pet)

# Access child nodes
for name, child in multi_pet_root.children.items():
    print(f"Child {name}: Total Sum (ns): {child.total_sum}")

Total Sum (ns): 220000
Total Sum (s): 0.00022
Total Mean (ns): 110000.0
Total Mean (s): 0.00011
Minimum Total (ns): 9223372036854775807
Minimum Total PET: -1
Maximum Total (ns): 0
Maximum Total PET: -1
Child child1: Total Sum (ns): 90000
Child child2: Total Sum (ns): 130000
